# Why is ECB flawed?

***By Jinan Jiang, contact jinan@berkeley.edu for corrections***

In [1]:
from crypto_demo_lib import *

In [28]:
#feel free to change these parameters
num_encs_per_display = 8

---

## ECB is deterministic

Without using IV to incorporate extra randomness in each encryption, ECB mode will output the same cipher text whenever you encrypt the same plaintext.  

Let's explore how this means we can lose IND-CPA when using ECB
![image info](./images/ECB_encryption.png)

In [29]:
message = b"a 16-byte string" #16 bytes
key = random_bytes(32)

print("encrypting the same message with the same key in ECB multiple times yield the same cipher text: \n")

for _ in range(num_encs_per_display):
    cypher_text = AES_ECB_enc(data = message, key = key)
    print(cypher_text.hex())

encrypting the same message with the same key in ECB multiple times yield the same cipher text: 

4029150c578b7203117e978fbfa50806
4029150c578b7203117e978fbfa50806
4029150c578b7203117e978fbfa50806
4029150c578b7203117e978fbfa50806
4029150c578b7203117e978fbfa50806
4029150c578b7203117e978fbfa50806
4029150c578b7203117e978fbfa50806
4029150c578b7203117e978fbfa50806


***we can use the following function to break IND-CPA***

In [23]:
def break_ind_cpa(mode):
    #let m0 and m1 start with same bytes
    m0 = b'1' * 16 + random_bytes(32)
    m1 = b'2' * 16 + random_bytes(32)
    
    ind_cpa = ind_cpa_game(mode)
    challange_text = ind_cpa.send_messages(m0, m1)
    
    cipher_text_m0 = ind_cpa.query_message(m0)
    if cipher_text_m0[:16] == challange_text[:16]:
        #if first block of the challenge ciphertext is the same as that of 
        #    m0, we know that m0 was encrypted
        return ind_cpa.is_guess_correct(m0)
    else:
        #otherwise, m1 was encrypted, and we guess m1
        return ind_cpa.is_guess_correct(m1)
    
def calc_success_rate(mode, num_tries):
    success_rate = 0
    for _ in range(100):
        if break_ind_cpa(mode):
            success_rate += 1
    return success_rate

In [24]:
#feel free to change these parameters
num_iterations = 5

In [25]:
print("Using the above method, the emperical chances of winning the IND-CPA game in the ECB mode are :")
for _ in range(num_iterations):
    success_rate = calc_success_rate(mode = modes.ECB, num_tries = 100)
    print (str(success_rate) + '%')

Using the above method, the emperical chances of winning the IND-CPA game in the ECB mode are :
100%
100%
100%
100%
100%


In contrast, let's look at CBC and CFB

**Even though the first block (16 bytes) of input into CBC and CFB modes are the same, the use of IV adds in extra randomness such that the block of ciphertexts corresponding to the same plaintexts will look different.**  

**Therefore, the above method of breaking IND-CPA won't work in CBC or CFB anymore:**

In [26]:
print("Using the above method, the emperical chances of winning the IND-CPA game in the CBC mode are :")
for _ in range(num_iterations):
    success_rate = calc_success_rate(mode = modes.CBC, num_tries = 100)
    print (str(success_rate) + '%')

Using the above method, the emperical chances of winning the IND-CPA game in the CBC mode are :
55%
50%
46%
49%
52%


**Which is about 50%; it didn't win the IND-CPA game!**

In [27]:
print("Using the above method, the emperical chances of winning the IND-CPA game in the CFB mode are :")
for _ in range(5):
    success_rate = calc_success_rate(mode = modes.CFB, num_tries = 100)
    print (str(success_rate) + '%')

Using the above method, the emperical chances of winning the IND-CPA game in the CFB mode are :
51%
48%
61%
50%
49%


---

## On the other hand, CBC is not

In [9]:
message = b"a 16-byte string" #16 bytes
key = random_bytes(32)

print("encrypting the same message with the same key in CBC multiple times: \n")

for _ in range(num_encs_per_display):
    cypher_text = AES_CBC_enc(data = message, key = key, iv = random_bytes(16))
    print(cypher_text.hex())

encrypting the same message with the same key in CBC multiple times: 

c363b0b8eed5ba9065d3d4fdc51d529e
17e3b3486718489c482645f16b04f0d6
acc40952b581102f8d6b06b8bcb65787
a1cf764953ddb30ca89666f0af91fe20
e2c1a5855ab94ab712e19bdec66d2dd2
3a089f0592d7362e8e1120a39a0b2143
ef57b955cd7af551a19b96b06967b833
6d934120974a7c0ee02b8430f5750094
a180b9ae6d372d049ce018c50902ce53
d50032275fae245ac8c7e318d4a8c2fc
25d6f34b8dc0cb697c1cab7a7096aa4a
5baeb4ea5bce3df16b40d1b621112c42
c0928bfc5068cd50453044844101b220
8d3a5f8d370681c9de1dfcc3a8094c1b
5105ee9087a19c8e21edbdba67c40ffe
6a8d4b088945431d816e1bee3e8cf2f3


---

## Let's also look at CFB if you are curious

In [10]:
message = b"a 16-byte string" #16 bytes
key = random_bytes(32)

print("encrypting the same message with the same key in CFB multiple times: \n")

for _ in range(num_encs_per_display):
    cypher_text = AES_CFB_enc(data = message, key = key, iv = random_bytes(16))
    print(cypher_text.hex())

encrypting the same message with the same key in CFB multiple times: 

e1fd2f7ac109c0ff69e680c7f1c1a23d
80df012b29c17d9c445f820dca0a268b
a83baf98e4a93e988e60dbf47cb7d534
fdc360f61d6ccc5be730ebe72ff236ab
1c571166c2cbde49c06c06bc09b5e6d0
285ee6b0353a75cfa7cd8155ccbccc51
feef74ef074f0d4694363e8978618ffc
1af699098f3d4d5b3d250e7af39717ed
fa799ea7839ab6a3165ac0ff3cfa49c5
65a3c9fd1620616a31685e17c2f46087
8a743073b3d06208faeb2dab0100fe89
16b182379b1034a085763a533245df9a
b36c990706d55eaf7d2a7774714aff7b
a6a6cc3afdc7a0b53b430acf269b86c2
0af15f6e897fcb691cc582049e862a2e
605e5e991d82c57d4f1897b185c40912
